In [ ]:
import os

def scale_input_data(scale_factor):
  file_bases = ['./input/avocado']
  for file_base in file_bases:
    import pandas as pd
    import shutil
    if scale_factor == 1.0:
      shutil.copyfile(file_base + '.csv', file_base + '.scaled.csv')
      continue
    df_to_scale = pd.read_csv(file_base + '.csv', index_col=0)
    new_num_rows = int(scale_factor * len(df_to_scale))
    if scale_factor <= 1.0:
      df_to_scale = df_to_scale.iloc[:new_num_rows]
    else:
      while len(df_to_scale) < new_num_rows:
        df_to_scale = pd.concat([df_to_scale, df_to_scale[:min(new_num_rows - len(df_to_scale), len(df_to_scale))]])
    df_to_scale.to_csv(file_base + '.scaled.csv', index_label='')

if 'INPUT_SCALE_FACTOR' in os.environ:
  scale_input_data(float(os.environ['INPUT_SCALE_FACTOR']))

### Analisis de Dataset de precios de paltas

Vamos a trabajar este kernel analizando un conjunto de datos y aplicando nuestro conocimiento de EDA asi como reconocimiento de funciones utiles de pandas y numpy

In [1]:
import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
exec(os.environ['IREWR_IMPORTS'])
# ALEX: remove plotting, ML code, path printing
# import seaborn as sns 

# from sklearn.metrics import mean_absolute_error
# from sklearn.preprocessing import LabelEncoder

# import plotly
# from plotly.offline import init_notebook_mode, iplot
# import plotly.graph_objs as go
# init_notebook_mode(connected=True)
# import matplotlib.pyplot as plt
# import pylab as pl
# sns.set(font_scale = 0.7)
# import os
# print(os.listdir("./input"))


### Visualizacion de informacion

![Paltas para todos](https://exoticfruitbox.com/wp-content/uploads/2015/10/aguacate.jpg)

## **Analisis de datos**

Para poder trabajar los datos primeros debemos conocer los datos y tipos de datos con los cuales vamos a trabajar, en este caso hemos tomado un Dataset de precios de paltas.

In [2]:
#Cargamos el data set mediante Python y Pandas mediante read_csv
#Tener en cuenta indicadar el separador como recomendacion con delimiter se recomienda
#En caso tengamos un campo donde se guarden los ID unico podemos caolocar elnumero de columna en index_col para usarlo de indexacion
data = pd.read_csv("./input/avocado.scaled.csv",delimiter=",",index_col=0)

#Visualizamos el volumen de filas y columnas de nuestros datos
data.shape

(18249, 13)

In [3]:
#Podemos mediante el comando info identificar los tipos de campos 123
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18249 entries, 0 to 11
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Date          18249 non-null  object 
 1   AveragePrice  18249 non-null  float64
 2   Total Volume  18249 non-null  float64
 3   4046          18249 non-null  float64
 4   4225          18249 non-null  float64
 5   4770          18249 non-null  float64
 6   Total Bags    18249 non-null  float64
 7   Small Bags    18249 non-null  float64
 8   Large Bags    18249 non-null  float64
 9   XLarge Bags   18249 non-null  float64
 10  type          18249 non-null  object 
 11  year          18249 non-null  int64  
 12  region        18249 non-null  object 
dtypes: float64(9), int64(1), object(3)
memory usage: 1.9+ MB


In [4]:
#Vamos a ver un pequeño resumen de los datos
data.head(5)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


Como bien indique tenemos que entender los datos y el significado de cada columna correctamente, solo de esa manera podemos avanzar cualquier paso siguiente.

En este caso que nos dice cada columna:

| Nombre | Descripcion  |
|---------|--------------|
| Date | Fecha de observacion  |
| AveragePrice | El precio promedio de cada palta  |
| type | El tipo (conventional or organic)  | 
| year | El año  | 
| Region | La ciudad o region observada  |  
| Total Volume | El volumen total comprado  |  
| 4046 | El numero total de paltas con codigo PLU 4046  |  
| 4225 | El numero total de paltas con codigo PLU 4225 sold  |  
| 4770 | El numero total de paltas con codigo PLU 4770 sold  |  
| Total Bags | El numero total de bolsas vendidas  |  
| Small Bags	| El numero total de bolsas pequeñas  |  
| Large Bags | El numero total de bolsas grandes  |  
| XLarge Bags | El numero total de bolsas extra grandes  |  

Cabe resaltar que

Total Volume = 4046 + 4225 + 4770 + Total Bags

Total Bags = Small Bags + Large Bags + XLarge Bags

Mas informacion de la data en [data](http://www.hassavocadoboard.com/retail/volume-and-price-data)

## **Consistencia de la informacion**

Luego de conocer la informacion vamos a analizar la consistencia de la informacion

In [5]:
#Vamos a validar que se cumple ambos formulas
#Todo aquel registro que no cumpla con la regla pasara a ser una inconsistencia la cual descartaremos.
dataIncorrecta = data[((data['4046'] + data['4225'] + data['4770'] + data['Total Bags']) != data['Total Volume'] ) | ((data['Small Bags'] + data['Large Bags'] + data['XLarge Bags']) != data['Total Bags'] )]
dataCorrecta =  data[((data['4046'] + data['4225'] + data['4770'] + data['Total Bags']) == data['Total Volume'] ) & ((data['Small Bags'] + data['Large Bags'] + data['XLarge Bags']) == data['Total Bags'] )]

dataIncorrecta.reset_index(inplace = True)
dataCorrecta.reset_index(inplace = True)

print("Data Total",data.shape)
print("Data Incorrecta",dataIncorrecta.shape)
print("Data Correcta",dataCorrecta.shape)

Data Total (18249, 13)
Data Incorrecta (8143, 14)
Data Correcta (10106, 14)


In [6]:
#Analizamos valores perdidos en caso de que existan
dataCorrecta.isnull().sum()

index           0
Date            0
AveragePrice    0
Total Volume    0
4046            0
4225            0
4770            0
Total Bags      0
Small Bags      0
Large Bags      0
XLarge Bags     0
type            0
year            0
region          0
dtype: int64

## **Enginner Features**
Vamos a realizar un grafico para extraer las anomalias o valores extremos para poder estandarizar lo mas posible nuestros datos

In [7]:
import itertools
# ALEX: remove plotting
# plt.subplots(figsize=(28,20))
time_spent=['Small Bags','Large Bags','XLarge Bags','XLarge Bags']
length=len(time_spent)
for i,j in itertools.zip_longest(time_spent,range(length)):
# ALEX: remove plotting
#     plt.subplot((length/2),2,j+1)
#     plt.subplots_adjust(wspace=0.2,hspace=0.5)
#     dataCorrecta[i].hist(bins=18,edgecolor='black')
#     plt.axvline(dataCorrecta[i].mean(),linestyle='dashed',color='r')
#     plt.title(i,size=20)
#     plt.xlabel('Tamaño de la bolsa')
#     plt.ylabel('cantidad de bolsas')
    dataCorrecta[i]
    _ = dataCorrecta[i].mean()
# ALEX: remove plotting
# plt.show()

In [8]:
dataCorrecta =dataCorrecta[(dataCorrecta['Small Bags']<300000.0) & (dataCorrecta['Large Bags']<150000.0) & (dataCorrecta['XLarge Bags']<10000.0)]
dataCorrecta.shape

(8888, 14)

In [9]:
import itertools
# ALEX: remove plotting
# plt.subplots(figsize=(25,16))
time_spent=['Small Bags','Large Bags','XLarge Bags','XLarge Bags']
length=len(time_spent)
for i,j in itertools.zip_longest(time_spent,range(length)):
# ALEX: remove plotting
#     plt.subplot((length/2),2,j+1)
#     plt.subplots_adjust(wspace=0.2,hspace=0.5)
#     dataCorrecta[i].hist(bins=18,edgecolor='black')
#     plt.axvline(dataCorrecta[i].mean(),linestyle='dashed',color='r')
#     plt.title(i,size=20)
#     plt.xlabel('Tamaño de la bolsa')
#     plt.ylabel('cantidad de bolsas')
    dataCorrecta[i]
    _ = dataCorrecta[i].mean()
# ALEX: remove plotting
# plt.show()

In [10]:
# ALEX: remove plotting
# pl.figure(figsize=(12,5))
# pl.title("Distribution Price")
# ax = sns.distplot(dataCorrecta["AveragePrice"], color = 'g')
_ = dataCorrecta["AveragePrice"]

In [11]:
# ALEX: remove plotting
# sns.boxplot(y="type", x="AveragePrice", data=dataCorrecta, palette = 'Set3')

In [12]:
conventional = dataCorrecta[dataCorrecta.type=="conventional"]
organic = dataCorrecta[dataCorrecta.type=="organic"]

groupBy1_price = conventional.groupby('Date').mean()
# ALEX: remove plotting
# scatter1 = go.Scatter(x=groupBy1_price.AveragePrice.index, y=groupBy1_price.AveragePrice, name="Conventional")
_ = groupBy1_price.AveragePrice.index
_ = groupBy1_price.AveragePrice

groupBy2_price = organic.groupby('Date').mean()
# ALEX: remove plotting
# scatter2 = go.Scatter(x=groupBy2_price.AveragePrice.index, y=groupBy2_price.AveragePrice, name="Organic")
_ = groupBy2_price.AveragePrice.index
_ = groupBy2_price.AveragePrice

# ALEX: remove plotting
# data = [scatter1, scatter2]
# layout=go.Layout(title="Time Series Plot for Mean Daily Price of Conventional and Organic Avocados", xaxis={'title':'Date'}, yaxis={'title':'Prices'})
# figure=go.Figure(data=data,layout=layout)
# iplot(figure)

/tmp/ipykernel_447864/801284835.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  groupBy1_price = conventional.groupby('Date').mean()
/tmp/ipykernel_447864/801284835.py:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  groupBy2_price = organic.groupby('Date').mean()


In [13]:
#dataCorrecta['Date2']=pd.to_datetime(dataCorrecta['Date'], format="%Y/%m/%d")
dataCorrecta['Date'] =dataCorrecta['Date'].astype('datetime64[ns]')



In [14]:
#Tenemos que analizar los datos categoricos no numericos, para transformarlos en numericos, o ver si son utiles
dataCorrecta['type'].value_counts()

organic         5495
conventional    3393
Name: type, dtype: int64

In [15]:
dataCorrecta.head()

,index,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
2,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany
3,5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,2015,Albany
4,7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,conventional,2015,Albany


In [16]:
dataCorrecta['year'].value_counts()

2015    2926
2016    2710
2017    2647
2018     605
Name: year, dtype: int64

In [17]:
dataCorrecta['region'].value_counts()

NorthernNewEngland     221
BuffaloRochester       218
Charlotte              214
Albany                 210
Jacksonville           207
Syracuse               205
Nashville              205
HarrisburgScranton     204
Spokane                200
Pittsburgh             200
Tampa                  199
Boston                 198
Boise                  194
Philadelphia           194
Sacramento             192
Roanoke                191
Orlando                191
GrandRapids            191
NewOrleansMobile       190
RaleighGreensboro      190
SouthCarolina          188
MiamiFtLauderdale      188
Louisville             186
SanFrancisco           186
LasVegas               184
RichmondNorfolk        184
StLouis                184
BaltimoreWashington    183
Chicago                180
Columbus               179
SanDiego               176
Atlanta                173
PhoenixTucson          173
HartfordSpringfield    169
Indianapolis           169
Houston                163
CincinnatiDayton       159
D

In [18]:
# ALEX: remove plotting
# fig, ax = plt.subplots(1, 1, figsize=(10,6))
# sns.boxplot(x='year',y='AveragePrice',data=dataCorrecta,color='red')

In [19]:
#Analizamos el comportamiento de los procesio por region de las paltas organicos
mask = dataCorrecta['type']=='organic'
# ALEX: remove plotting
# g = sns.factorplot('AveragePrice','region',data=dataCorrecta[mask],
#                    hue='year',
#                    height=13,
#                    aspect=0.8,
#                    palette='magma',
#                    join=False,
#               )
_ = dataCorrecta[mask]

In [20]:
#Analizamos el comportamiento de los procesio por region de las paltas convencionales
mask = dataCorrecta['type']=='conventional'
# ALEX: remove plotting
# g = sns.factorplot('AveragePrice','region',data=dataCorrecta[mask],
#                    hue='year',
#                    height=13,
#                    aspect=0.8,
#                    palette='magma',
#                    join=False,
#               )
_ = dataCorrecta[mask]

In [21]:
# ALEX: remove ML code
# label = LabelEncoder()
# dicts = {}

# label.fit(dataCorrecta.type.drop_duplicates()) 
# dicts['type'] = list(label.classes_)
# dataCorrecta.type = label.transform(dataCorrecta.type) 

In [22]:
dataCorrecta['type'].value_counts()

organic         5495
conventional    3393
Name: type, dtype: int64

In [23]:
#Realizamos una tabla de correlacion, para conocer el nivel de relacion entre los campos y entre nuestro target
cols = ['AveragePrice','Total Volume','4046','4225','4770','Total Bags','Small Bags','Large Bags','XLarge Bags','year','type']
# ALEX: remove plotting
# sns.set(font_scale = 1.5)
corr = dataCorrecta[cols].corr('spearman')
# ALEX: remove plotting
# plt.figure(figsize = ( 14 , 14 )) 
# sns.heatmap(corr,annot=True,fmt='.2f',cmap="YlGnBu");

/tmp/ipykernel_447864/1994942820.py:5: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr = dataCorrecta[cols].corr('spearman')


In [24]:
#Creamos las variables ficticias para region
region_dummi =pd.get_dummies(dataCorrecta['region'], prefix='reg')
region_dummi.head()

,reg_Albany,reg_Atlanta,reg_BaltimoreWashington,reg_Boise,reg_Boston,reg_BuffaloRochester,reg_California,reg_Charlotte,reg_Chicago,reg_CincinnatiDayton,...,reg_SouthCarolina,reg_SouthCentral,reg_Southeast,reg_Spokane,reg_StLouis,reg_Syracuse,reg_Tampa,reg_TotalUS,reg_West,reg_WestTexNewMexico
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
#Agregamos los datos fictios a la data original
data_nueva = pd.concat([dataCorrecta, region_dummi], axis=1)
data_nueva.head() 

,index,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,...,reg_SouthCarolina,reg_SouthCentral,reg_Southeast,reg_Spokane,reg_StLouis,reg_Syracuse,reg_Tampa,reg_TotalUS,reg_West,reg_WestTexNewMexico
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,...,0,0,0,0,0,0,0,0,0,0
1,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,...,0,0,0,0,0,0,0,0,0,0
2,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,...,0,0,0,0,0,0,0,0,0,0
3,5,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,...,0,0,0,0,0,0,0,0,0,0
4,7,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#eliminamos las columna region
data_nueva = data_nueva.drop('region', 1)
data_nueva = data_nueva.drop('index', 1)

/tmp/ipykernel_447864/3097472254.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_nueva = data_nueva.drop('region', 1)
/tmp/ipykernel_447864/3097472254.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  data_nueva = data_nueva.drop('index', 1)


Generamos un grafico de correlacion

In [27]:
#corr = data_nueva.corr('spearman') 
#plt.figure(figsize = ( 35 , 20 )) 
#sns.heatmap(corr,annot=True,fmt='.2f',cmap="YlGnBu");

In [28]:
data_train = pd.DataFrame(index=data_nueva.index)
targer_train = pd.DataFrame(index=data_nueva.index)
data_train = data_nueva
targer_train = data_nueva['AveragePrice']

In [29]:
data_train['monthy'] = data_train['Date'].astype('datetime64[ns]').apply(lambda ts: ts.month)

In [30]:
dummi_month =pd.get_dummies(data_train['monthy'], prefix='month')
data_train = pd.concat([data_train, dummi_month], axis=1)

In [31]:
data_train = data_train.drop(['monthy'],axis=1)
#data_train = data_train.drop(['Date'],axis=1)

In [32]:
data_train.head(5)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,...,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,...,0,0,0,0,0,0,0,0,0,1
1,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,...,0,0,0,0,0,0,0,0,1,0
3,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,...,0,0,0,0,0,0,0,0,1,0
4,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,...,0,0,0,0,0,0,0,0,1,0


In [33]:
data_train['year'].value_counts()

2015    2926
2016    2710
2017    2647
2018     605
Name: year, dtype: int64

In [34]:
region_dummi_fecha =pd.get_dummies(data_train['year'], prefix='year')

In [35]:
data_train = pd.concat([data_train, region_dummi_fecha], axis=1)

In [36]:
data_train.head(5)

,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,...,month_7,month_8,month_9,month_10,month_11,month_12,year_2015,year_2016,year_2017,year_2018
0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,...,0,0,0,0,0,1,1,0,0,0
1,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,...,0,0,0,0,0,1,1,0,0,0
2,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,...,0,0,0,0,1,0,1,0,0,0
3,2015-11-22,1.26,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,...,0,0,0,0,1,0,1,0,0,0
4,2015-11-08,0.98,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,...,0,0,0,0,1,0,1,0,0,0


In [37]:
data_train = data_train.drop(['year'],axis=1)
data_train = data_train.drop(['Date'], axis=1)
data_train = data_train.drop(['AveragePrice'],axis=1)

In [38]:

data_train.head(5)

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,reg_Albany,...,month_7,month_8,month_9,month_10,month_11,month_12,year_2015,year_2016,year_2017,year_2018
0,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,1,...,0,0,0,0,0,1,1,0,0,0
1,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,1,...,0,0,0,0,0,1,1,0,0,0
2,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0
3,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0
4,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0


In [39]:
targer_train.mean()

1.4789187668766877

In [40]:
# ALEX: remove plotting
# %config InlineBackend.figure_format = 'svg'
# sns.set(font_scale = 1)
# sns.distplot(targer_train);

In [41]:
# ALEX: remove plotting
# sns.set(font_scale = 1)
# sns.distplot(np.log1p(targer_train));
_ = np.log1p(targer_train)

Creamos la separacion entre la data de test y entrenamiento

In [42]:
data_train.head()

,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,reg_Albany,...,month_7,month_8,month_9,month_10,month_11,month_12,year_2015,year_2016,year_2017,year_2018
0,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,1,...,0,0,0,0,0,1,1,0,0,0
1,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,1,...,0,0,0,0,0,1,1,0,0,0
2,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0
3,55979.78,1184.27,48067.99,43.61,6683.91,6556.47,127.44,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0
4,109428.33,703.75,101815.36,80.00,6829.22,6266.85,562.37,0.0,conventional,1,...,0,0,0,0,1,0,1,0,0,0


In [43]:
# ALEX: remove ML code
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(data_train,targer_train,test_size=0.2)
# print(X_train.shape,y_train.shape)
# print(X_test.shape,y_test.shape)

## Luego de este punto podemos trabajarlo con un modelo de prediccion
De acuerdo con el modelo de prediccion podemos trabajar los datos para predecir el precio promedio individual, empezamos con el RIDGE

In [44]:
#Probamos con un modelo simple Ridge
# ALEX: remove ML code
# from sklearn.linear_model import Ridge
# ridge = Ridge(random_state=17)
# ridge.fit(X_train, y_train);
# ridge_pred = ridge.predict(X_test)

In [45]:
# ALEX: remove plotting
# sns.set(font_scale = 1)
# plt.hist(y_test, bins=50, alpha=.5, color='red', label='true', range=(0,4));
# plt.hist(ridge_pred, bins=50, alpha=.5, color='green', label='pred', range=(0,4));
# plt.legend();

In [46]:
# ALEX: remove ML code
# mean_absolute_error(y_test, ridge_pred)

Probamos con varios modelos para determinar el que nos da mejores resultados con la configuracion estandar

In [47]:
# ALEX: remove ML code
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.svm import SVR
# from sklearn.ensemble import AdaBoostRegressor
# from sklearn.ensemble import GradientBoostingRegressor
# from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
# from sklearn.linear_model import Lasso,Ridge,BayesianRidge,ElasticNet,HuberRegressor,LinearRegression,LogisticRegression,SGDRegressor
# from sklearn.metrics import mean_squared_error
# import numpy as np
# import warnings
# warnings.filterwarnings("ignore")

# # print X_train.shape, X_test.shape

# classifiers = [['DecisionTree :',DecisionTreeRegressor()],
#                ['RandomForest :',RandomForestRegressor()],
#                ['KNeighbours :', KNeighborsRegressor(n_neighbors = 2)],
#                ['SVM :', SVR()],
#                ['AdaBoostClassifier :', AdaBoostRegressor()],
#                ['GradientBoostingClassifier: ', GradientBoostingRegressor()],
#                ['Xgboost: ', XGBRegressor()],
#                ['CatBoost: ', CatBoostRegressor(logging_level='Silent')],
#                ['Lasso: ', Lasso()],
#                ['Ridge: ', Ridge(random_state=17)],
#                ['BayesianRidge: ', BayesianRidge()],
#                ['ElasticNet: ', ElasticNet()],
#                ['HuberRegressor: ', HuberRegressor()]]

# print("Accuracy Results...")


# for name,classifier in classifiers:
#     classifier = classifier
#     classifier.fit(X_train, y_train)
#     predictions = classifier.predict(X_test)
#     print(name, (np.sqrt(mean_squared_error(y_test, predictions))))

En este caso vamos a tomar el RandomForest y lo probamos de manera individual con el MAE

In [48]:
# ALEX: remove ML code
# classifier = RandomForestRegressor()
# classifier.fit(X_train, y_train)
# predictions = classifier.predict(X_test)


In [49]:
# ALEX: remove ML code
# mean_absolute_error(y_test, predictions)

In [50]:
# ALEX: remove ML code
# sns.set(font_scale = 1)
# data = pd.DataFrame({'Y Test':y_test , 'Pred':predictions},columns=['Y Test','Pred'])
# sns.lmplot(x='Y Test',y='Pred',data=data,palette='rainbow')
# data.head()